# Natural Language Processing

# Retrieval-Augmented generation (RAG)

1. Prompt
2. Retrieval
3. Memory
4. Chain

In [1]:
import os
import torch
# Set GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

os.environ['http_proxy']  = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [2]:
from langchain import PromptTemplate

prompt_template = """
    Hello! I’m your trusty PattyBot, here to lend a hand to you and your friends with any questions or tasks you’ve got! 
    Whether you’re puzzling over life’s big mysteries, need help with a project, or just want some practical advice, 
    I’m here to break things down and offer clear, helpful answers. 
    No topic’s too big or small—just let me know what’s on your mind, and I’ll do my best to assist you!
    {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format 
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Hello! I’m your trusty PattyBot, here to lend a hand to you and your friends with any questions or tasks you’ve got! \n    Whether you’re puzzling over life’s big mysteries, need help with a project, or just want some practical advice, \n    I’m here to break things down and offer clear, helpful answers. \n    No topic’s too big or small—just let me know what’s on your mind, and I’ll do my best to assist you!\n    {context}\n    Question: {question}\n    Answer:')

In [3]:
PROMPT.format(
    context = "My name is Patsachon, you can called me Natasha",
    question = "What is your name?"
)

'Hello! I’m your trusty PattyBot, here to lend a hand to you and your friends with any questions or tasks you’ve got! \n    Whether you’re puzzling over life’s big mysteries, need help with a project, or just want some practical advice, \n    I’m here to break things down and offer clear, helpful answers. \n    No topic’s too big or small—just let me know what’s on your mind, and I’ll do my best to assist you!\n    My name is Patsachon, you can called me Natasha\n    Question: What is your name?\n    Answer:'

Note : [How to improve prompting (Zero-shot, Few-shot, Chain-of-Thought, etc.](https://github.com/chaklam-silpasuwanchai/Natural-Language-Processing/blob/main/Code/05%20-%20RAG/advance/cot-tot-prompting.ipynb)

## 2. Retrieval

1. `Document loaders` : Load documents from many different sources (HTML, PDF, code). 
2. `Document transformers` : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. `Text embedding models` : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. `Vector stores`: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. `Retrievers` : Once the data is in the database, you still need to retrieve it.

### 2.1 Document Loaders 
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

[PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

[Download Document](https://web.stanford.edu/~jurafsky/slp3/)

In [4]:
from langchain.document_loaders import PyMuPDFLoader

nlp_docs = './Personal_Info-4.pdf'

loader = PyMuPDFLoader(nlp_docs)
documents = loader.load()

In [5]:
# documents

In [6]:
len(documents)

1

In [7]:
documents[0]

Document(metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': './Personal_Info-4.pdf', 'file_path': './Personal_Info-4.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Personal_Info', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content="PERSONAL INFORMATION \nAge: 23 years old \nEDUCATION \nCurrent: Master's Degree (In Progress) \n●\u200b Field: Data Science and AI \n●\u200b Institution: AIT (Asian Institute of Technology) \nCompleted: Bachelor's Degree \n●\u200b Major: Financial Engineering \n●\u200b Institution: KMITL \nPROFESSIONAL EXPERIENCE \nPosition: Governance Researcher \n●\u200b Industry: Cryptocurrency \n●\u200b work experience: 3 years \n●\u200b Responsibilities:Researched cryptocurrency governance policies and analyzed \ndecision-making processes in the crypto space \nRESEARCH INTERESTS \n●\u200b Exploring decentralized finance (DeFi)

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [9]:
doc[0]

Document(metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': './Personal_Info-4.pdf', 'file_path': './Personal_Info-4.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Personal_Info', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content="PERSONAL INFORMATION \nAge: 23 years old \nEDUCATION \nCurrent: Master's Degree (In Progress) \n●\u200b Field: Data Science and AI \n●\u200b Institution: AIT (Asian Institute of Technology) \nCompleted: Bachelor's Degree \n●\u200b Major: Financial Engineering \n●\u200b Institution: KMITL \nPROFESSIONAL EXPERIENCE \nPosition: Governance Researcher \n●\u200b Industry: Cryptocurrency \n●\u200b work experience: 3 years \n●\u200b Responsibilities:Researched cryptocurrency governance policies and analyzed \ndecision-making processes in the crypto space \nRESEARCH INTERESTS \n●\u200b Exploring decentralized finance (DeFi)

In [10]:
len(doc)

2

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

*Note* Instructor Model : [Huggingface](gingface.co/hkunlp/instructor-base) | [Paper](https://arxiv.org/abs/2212.09741)

In [11]:
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name=model_name,
    model_kwargs={"device": device}
)


/home/jupyter-st124952/.local/lib/python3.12/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/home/jupyter-st124952/.local/lib/python3.12/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


load INSTRUCTOR_Transformer


/home/jupyter-st124952/.local/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/jupyter-st124952/.local/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


max_seq_length  512


### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [12]:
#locate vectorstore
vector_path = './vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

In [13]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'nlp_stanford'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'nlp' #default index
)

### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [14]:
#calling vector from local
vector_path = './vector-store'
db_file_name = 'nlp_stanford'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp', #default index
    allow_dangerous_deserialization=True
)  

In [15]:
#ready to use
retriever = vectordb.as_retriever()

In [16]:
retriever.get_relevant_documents("How old are you?")

/tmp/ipykernel_7549/342014793.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("How old are you?")


[Document(id='1fd97ff3-8228-4bc6-94b2-bbcb0ed72c27', metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': './Personal_Info-4.pdf', 'file_path': './Personal_Info-4.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Personal_Info', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content="PERSONAL INFORMATION \nAge: 23 years old \nEDUCATION \nCurrent: Master's Degree (In Progress) \n●\u200b Field: Data Science and AI \n●\u200b Institution: AIT (Asian Institute of Technology) \nCompleted: Bachelor's Degree \n●\u200b Major: Financial Engineering \n●\u200b Institution: KMITL \nPROFESSIONAL EXPERIENCE \nPosition: Governance Researcher \n●\u200b Industry: Cryptocurrency \n●\u200b work experience: 3 years \n●\u200b Responsibilities:Researched cryptocurrency governance policies and analyzed \ndecision-making processes in the crypto space \nRESEARCH INTERESTS \n●\

In [17]:
retriever.get_relevant_documents("What is your highest level of education?")

[Document(id='1fd97ff3-8228-4bc6-94b2-bbcb0ed72c27', metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': './Personal_Info-4.pdf', 'file_path': './Personal_Info-4.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Personal_Info', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content="PERSONAL INFORMATION \nAge: 23 years old \nEDUCATION \nCurrent: Master's Degree (In Progress) \n●\u200b Field: Data Science and AI \n●\u200b Institution: AIT (Asian Institute of Technology) \nCompleted: Bachelor's Degree \n●\u200b Major: Financial Engineering \n●\u200b Institution: KMITL \nPROFESSIONAL EXPERIENCE \nPosition: Governance Researcher \n●\u200b Industry: Cryptocurrency \n●\u200b work experience: 3 years \n●\u200b Responsibilities:Researched cryptocurrency governance policies and analyzed \ndecision-making processes in the crypto space \nRESEARCH INTERESTS \n●\

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [18]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

InMemoryChatMessageHistory(messages=[])

In [19]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [20]:
history

InMemoryChatMessageHistory(messages=[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Whats up?', additional_kwargs={}, response_metadata={}), HumanMessage(content='How are you', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm quite good. How about you?", additional_kwargs={}, response_metadata={})])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios. 
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [21]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

/tmp/ipykernel_7549/1450517278.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [22]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content="What's up?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm quite good. How about you?", additional_kwargs={}, response_metadata={})]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time. 
- it only uses the last K interactions. 
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [23]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

/tmp/ipykernel_7549/1660141487.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available), 
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [24]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch

model_id = 'lmsys/fastchat-t5-3b-v1.0'

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id

bitsandbyte_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    quantization_config = bitsandbyte_config, #caution Nvidia
    device_map = 'auto',
    load_in_8bit = True
)

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 256,
    model_kwargs = {
        "temperature" : 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

/home/jupyter-st124952/.local/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained

### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [25]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [26]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], input_types={}, partial_variables={}, template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [27]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

/tmp/ipykernel_7549/3952007952.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  question_generator = LLMChain(


In [28]:
query = 'Comparing both of them'
chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})

/tmp/ipykernel_7549/3338227403.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  question_generator({'chat_history' : chat_history, "question" : query})




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is Machine Learning
AI:
Human:What is Deep Learning
AI:
Follow Up Input: Comparing both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:',
 'question': 'Comparing both of them',
 'text': '<pad> What  is  the  difference  between  Machine  Learning  and  Deep  Learning  AI?\n'}

`combine_docs_chain`

In [29]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

/tmp/ipykernel_7549/1999184610.py:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  doc_chain = load_qa_chain(


StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Hello! I’m your trusty PattyBot, here to lend a hand to you and your friends with any questions or tasks you’ve got! \n    Whether you’re puzzling over life’s big mysteries, need help with a project, or just want some practical advice, \n    I’m here to break things down and offer clear, helpful answers. \n    No topic’s too big or small—just let me know what’s on your mind, and I’ll do my best to assist you!\n    {context}\n    Question: {question}\n    Answer:'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x75e167e8a240>), output_parser=StrOutputParser(), llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_variable_name='context')

In [30]:
query = "How old are you"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Hello! I’m your trusty PattyBot, here to lend a hand to you and your friends with any questions or tasks you’ve got! 
    Whether you’re puzzling over life’s big mysteries, need help with a project, or just want some practical advice, 
    I’m here to break things down and offer clear, helpful answers. 
    No topic’s too big or small—just let me know what’s on your mind, and I’ll do my best to assist you!
    PERSONAL INFORMATION 
Age: 23 years old 
EDUCATION 
Current: Master's Degree (In Progress) 
●​ Field: Data Science and AI 
●​ Institution: AIT (Asian Institute of Technology) 
Completed: Bachelor's Degree 
●​ Major: Financial Engineering 
●​ Institution: KMITL 
PROFESSIONAL EXPERIENCE 
Position: Governance Researcher 
●​ Industry: Cryptocurrency 
●​ work experience: 3 years 
●​ Responsibilities:Researched cryptocurrency governance policies and analyzed 
decision-making proces

{'input_documents': [Document(id='1fd97ff3-8228-4bc6-94b2-bbcb0ed72c27', metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': './Personal_Info-4.pdf', 'file_path': './Personal_Info-4.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Personal_Info', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content="PERSONAL INFORMATION \nAge: 23 years old \nEDUCATION \nCurrent: Master's Degree (In Progress) \n●\u200b Field: Data Science and AI \n●\u200b Institution: AIT (Asian Institute of Technology) \nCompleted: Bachelor's Degree \n●\u200b Major: Financial Engineering \n●\u200b Institution: KMITL \nPROFESSIONAL EXPERIENCE \nPosition: Governance Researcher \n●\u200b Industry: Cryptocurrency \n●\u200b work experience: 3 years \n●\u200b Responsibilities:Researched cryptocurrency governance policies and analyzed \ndecision-making processes in the crypto space \nRES

In [31]:
memory = ConversationBufferWindowMemory(
    k=3, 
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

/tmp/ipykernel_7549/4136652467.py:8: LangChainDeprecationWarning: The class `ConversationalRetrievalChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~create_history_aware_retriever together with create_retrieval_chain (see example in docstring)` instead.
  chain = ConversationalRetrievalChain(


ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Hello! I’m your trusty PattyBot, here to lend a hand to you and your friends with any questions or tasks you’ve got! \n    Whether you’re puzzling over life’s big mysteries, need help with a project, or just want some practical advice, \n    I’m here to break things down and offer clear, helpful answers. \n    No topic’s too big or small—just let me know what’s on your mind, and I’ll do my best to assist you!\n    {context}\n    Question: {question}\n    Answer:'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x75e16

## 5. Chatbot

In [32]:
import re
import json

qa_pairs = []

def clean_answer(answer):
    answer = re.sub(r'<pad>', '', answer)
    answer = re.sub(r'pad>', '', answer)
    answer = re.sub(r'\s+', ' ', answer)
    answer = answer.strip()
    return answer


In [33]:
prompt_question = "How old are you?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Hello! I’m your trusty PattyBot, here to lend a hand to you and your friends with any questions or tasks you’ve got! 
    Whether you’re puzzling over life’s big mysteries, need help with a project, or just want some practical advice, 
    I’m here to break things down and offer clear, helpful answers. 
    No topic’s too big or small—just let me know what’s on your mind, and I’ll do my best to assist you!
    PERSONAL INFORMATION 
Age: 23 years old 
EDUCATION 
Current: Master's Degree (In Progress) 
●​ Field: Data Science and AI 
●​ Institution: AIT (Asian Institute of Technology) 
Completed: Bachelor's Degree 
●​ Major: Financial Engineering 
●​ Institution: KMITL 
PROFESSIONAL EXPERIENCE 
Position: Governance Researcher 
●​ Industry: Cryptocurrency 
●​ work experience: 3 years 
●​ Responsibilities:Researched cryptocurrency g

{'question': 'How old are you?',
 'chat_history': [],
 'answer': '<pad>  23  years  old.\n',
 'source_documents': [Document(id='1fd97ff3-8228-4bc6-94b2-bbcb0ed72c27', metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': './Personal_Info-4.pdf', 'file_path': './Personal_Info-4.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Personal_Info', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content="PERSONAL INFORMATION \nAge: 23 years old \nEDUCATION \nCurrent: Master's Degree (In Progress) \n●\u200b Field: Data Science and AI \n●\u200b Institution: AIT (Asian Institute of Technology) \nCompleted: Bachelor's Degree \n●\u200b Major: Financial Engineering \n●\u200b Institution: KMITL \nPROFESSIONAL EXPERIENCE \nPosition: Governance Researcher \n●\u200b Industry: Cryptocurrency \n●\u200b work experience: 3 years \n●\u200b Responsibilities:Researched cryptoc

In [34]:
cleaned_answer = clean_answer(answer['answer'])
qa_pair = {
    "question": prompt_question,
    "answer": cleaned_answer 
}
qa_pairs.append(qa_pair)

In [35]:
prompt_question = "What is your highest level of education?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='How old are you?', additional_kwargs={}, response_metadata={}), AIMessage(content='<pad>  23  years  old.\n', additional_kwargs={}, response_metadata={})]
Follow Up Input: What is your highest level of education?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Hello! I’m your trusty PattyBot, here to lend a hand to you and your friends with any questions or tasks you’ve got! 
    Whether you’re puzzling over life’s big mysteries, need help with a project, or just want some practical advice, 
    I’m here to break things down and offer clear, helpful answers. 
    No topic’s too big or small—just 

{'question': 'What is your highest level of education?',
 'chat_history': [HumanMessage(content='How old are you?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<pad>  23  years  old.\n', additional_kwargs={}, response_metadata={})],
 'answer': "<pad>  Master's  Degree  (In  Progress)\n",
 'source_documents': [Document(id='1fd97ff3-8228-4bc6-94b2-bbcb0ed72c27', metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': './Personal_Info-4.pdf', 'file_path': './Personal_Info-4.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Personal_Info', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content="PERSONAL INFORMATION \nAge: 23 years old \nEDUCATION \nCurrent: Master's Degree (In Progress) \n●\u200b Field: Data Science and AI \n●\u200b Institution: AIT (Asian Institute of Technology) \nCompleted: Bachelor's Degree \n●\u200b Major: Financia

In [36]:
cleaned_answer = clean_answer(answer['answer'])
qa_pair = {
    "question": prompt_question,
    "answer": cleaned_answer 
}
qa_pairs.append(qa_pair)

In [37]:
prompt_question = "What major or field of study did you pursue during your education?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='How old are you?', additional_kwargs={}, response_metadata={}), AIMessage(content='<pad>  23  years  old.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your highest level of education?', additional_kwargs={}, response_metadata={}), AIMessage(content="<pad>  Master's  Degree  (In  Progress)\n", additional_kwargs={}, response_metadata={})]
Follow Up Input: What major or field of study did you pursue during your education?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Hello! I’m your trusty PattyBot, here to lend a hand to you and your friends with any questions or

{'question': 'What major or field of study did you pursue during your education?',
 'chat_history': [HumanMessage(content='How old are you?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<pad>  23  years  old.\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is your highest level of education?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="<pad>  Master's  Degree  (In  Progress)\n", additional_kwargs={}, response_metadata={})],
 'answer': '<pad>  Financial  Engineering\n',
 'source_documents': [Document(id='1fd97ff3-8228-4bc6-94b2-bbcb0ed72c27', metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': './Personal_Info-4.pdf', 'file_path': './Personal_Info-4.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Personal_Info', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content="PERSONAL

In [38]:
cleaned_answer = clean_answer(answer['answer'])
qa_pair = {
    "question": prompt_question,
    "answer": cleaned_answer 
}
qa_pairs.append(qa_pair)

In [39]:
prompt_question = "How many years of work experience do you have?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='How old are you?', additional_kwargs={}, response_metadata={}), AIMessage(content='<pad>  23  years  old.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your highest level of education?', additional_kwargs={}, response_metadata={}), AIMessage(content="<pad>  Master's  Degree  (In  Progress)\n", additional_kwargs={}, response_metadata={}), HumanMessage(content='What major or field of study did you pursue during your education?', additional_kwargs={}, response_metadata={}), AIMessage(content='<pad>  Financial  Engineering\n', additional_kwargs={}, response_metadata={})]
Follow Up Input: How many years of work experience do you have?
Standalone question:

> Finish

{'question': 'How many years of work experience do you have?',
 'chat_history': [HumanMessage(content='How old are you?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<pad>  23  years  old.\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is your highest level of education?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="<pad>  Master's  Degree  (In  Progress)\n", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What major or field of study did you pursue during your education?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<pad>  Financial  Engineering\n', additional_kwargs={}, response_metadata={})],
 'answer': '<pad>  3  years\n',
 'source_documents': [Document(id='1fd97ff3-8228-4bc6-94b2-bbcb0ed72c27', metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': './Personal_Info-4.pdf', 'file_path': './Personal_Info-4.pdf', 'tota

In [40]:
cleaned_answer = clean_answer(answer['answer'])
qa_pair = {
    "question": prompt_question,
    "answer": cleaned_answer 
}
qa_pairs.append(qa_pair)

In [41]:
prompt_question = "What type of work or industry have you been involved in?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='What is your highest level of education?', additional_kwargs={}, response_metadata={}), AIMessage(content="<pad>  Master's  Degree  (In  Progress)\n", additional_kwargs={}, response_metadata={}), HumanMessage(content='What major or field of study did you pursue during your education?', additional_kwargs={}, response_metadata={}), AIMessage(content='<pad>  Financial  Engineering\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='How many years of work experience do you have?', additional_kwargs={}, response_metadata={}), AIMessage(content='<pad>  3  years\n', additional_kwargs={}, response_metadata={})]
Follow Up Input: What type of work or industry have you been involved i

/home/jupyter-st124952/.local/lib/python3.12/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


{'question': 'What type of work or industry have you been involved in?',
 'chat_history': [HumanMessage(content='What is your highest level of education?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="<pad>  Master's  Degree  (In  Progress)\n", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What major or field of study did you pursue during your education?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<pad>  Financial  Engineering\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How many years of work experience do you have?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<pad>  3  years\n', additional_kwargs={}, response_metadata={})],
 'answer': '<pad>  Governance  Researcher  in  the  cryptocurrency  industry.\n',
 'source_documents': [Document(id='1fd97ff3-8228-4bc6-94b2-bbcb0ed72c27', metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': '', 'creationdate':

In [42]:
cleaned_answer = clean_answer(answer['answer'])
qa_pair = {
    "question": prompt_question,
    "answer": cleaned_answer 
}
qa_pairs.append(qa_pair)

In [43]:
prompt_question = "Can you describe your current role or job responsibilities?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='What major or field of study did you pursue during your education?', additional_kwargs={}, response_metadata={}), AIMessage(content='<pad>  Financial  Engineering\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='How many years of work experience do you have?', additional_kwargs={}, response_metadata={}), AIMessage(content='<pad>  3  years\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='What type of work or industry have you been involved in?', additional_kwargs={}, response_metadata={}), AIMessage(content='<pad>  Governance  Researcher  in  the  cryptocurrency  industry.\n', additional_kwargs={}, response_metadata={})]
Follow Up Input: Can you desc

/home/jupyter-st124952/.local/lib/python3.12/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Hello! I’m your trusty PattyBot, here to lend a hand to you and your friends with any questions or tasks you’ve got! 
    Whether you’re puzzling over life’s big mysteries, need help with a project, or just want some practical advice, 
    I’m here to break things down and offer clear, helpful answers. 
    No topic’s too big or small—just let me know what’s on your mind, and I’ll do my best to assist you!
    PERSONAL INFORMATION 
Age: 23 years old 
EDUCATION 
Current: Master's Degree (In Progress) 
●​ Field: Data Science and AI 
●​ Institution: AIT (Asian Institute of Technology) 
Completed: Bachelor's Degree 
●​ Major: Financial Engineering 
●​ Institution: KMITL 
PROFESSIONAL EXPERIENCE 
Position: Governance Researcher 
●​ Industry: Cryptocurrency 
●​ work experience: 3 years 
●​ Responsibilities:Researched cryptocurrency governance policies and analyzed 
dec

/home/jupyter-st124952/.local/lib/python3.12/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


{'question': 'Can you describe your current role or job responsibilities?',
 'chat_history': [HumanMessage(content='What major or field of study did you pursue during your education?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<pad>  Financial  Engineering\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How many years of work experience do you have?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<pad>  3  years\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What type of work or industry have you been involved in?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<pad>  Governance  Researcher  in  the  cryptocurrency  industry.\n', additional_kwargs={}, response_metadata={})],
 'answer': '<pad> < pad>  My  current  role  is  a  Governance  Researcher  in  the  Cryptocurrency  industry.  My  job  responsibilities  include  researching  cryptocurrency  governance  policies  an

In [44]:
cleaned_answer = clean_answer(answer['answer'])
qa_pair = {
    "question": prompt_question,
    "answer": cleaned_answer 
}
qa_pairs.append(qa_pair)

In [45]:
prompt_question = "What are your core beliefs regarding the role of technology in shaping society?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='How many years of work experience do you have?', additional_kwargs={}, response_metadata={}), AIMessage(content='<pad>  3  years\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='What type of work or industry have you been involved in?', additional_kwargs={}, response_metadata={}), AIMessage(content='<pad>  Governance  Researcher  in  the  cryptocurrency  industry.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Can you describe your current role or job responsibilities?', additional_kwargs={}, response_metadata={}), AIMessage(content='<pad> < pad>  My  current  role  is  a  Governance  Researcher  in  the  Cryptocurrency  industry.  My  job  respon

/home/jupyter-st124952/.local/lib/python3.12/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Hello! I’m your trusty PattyBot, here to lend a hand to you and your friends with any questions or tasks you’ve got! 
    Whether you’re puzzling over life’s big mysteries, need help with a project, or just want some practical advice, 
    I’m here to break things down and offer clear, helpful answers. 
    No topic’s too big or small—just let me know what’s on your mind, and I’ll do my best to assist you!
    ●​ Integrating cryptocurrency knowledge with financial models 
PHILOSOPHY 
●​ Believes technology should democratize opportunities while maintaining fairness and 
sustainability 
●​ Advocates for cultural values shaping technological development to respect diversity, 
privacy, and community needs 
ACADEMIC CHALLENGES 
●​ Connecting theoretical concepts to real-time market dynamics in Financial 
Engineering

PERSONAL INFORMATION 
Age: 23 years old 
EDUCATION

/home/jupyter-st124952/.local/lib/python3.12/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


{'question': 'What are your core beliefs regarding the role of technology in shaping society?',
 'chat_history': [HumanMessage(content='How many years of work experience do you have?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<pad>  3  years\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What type of work or industry have you been involved in?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<pad>  Governance  Researcher  in  the  cryptocurrency  industry.\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Can you describe your current role or job responsibilities?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<pad> < pad>  My  current  role  is  a  Governance  Researcher  in  the  Cryptocurrency  industry.  My  job  responsibilities  include  researching  cryptocurrency  governance  policies  and  analyzing  decision-making  processes  in  the  crypto  space.\n', additiona

In [46]:
cleaned_answer = clean_answer(answer['answer'])
qa_pair = {
    "question": prompt_question,
    "answer": cleaned_answer 
}
qa_pairs.append(qa_pair)

In [47]:
prompt_question = "How do you think cultural values should influence technological advancements?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='What type of work or industry have you been involved in?', additional_kwargs={}, response_metadata={}), AIMessage(content='<pad>  Governance  Researcher  in  the  cryptocurrency  industry.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Can you describe your current role or job responsibilities?', additional_kwargs={}, response_metadata={}), AIMessage(content='<pad> < pad>  My  current  role  is  a  Governance  Researcher  in  the  Cryptocurrency  industry.  My  job  responsibilities  include  researching  cryptocurrency  governance  policies  and  analyzing  decision-making  processes  in  the  crypto  space.\n', additional_kwargs={}, response_metadata={}), HumanMessag

/home/jupyter-st124952/.local/lib/python3.12/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Hello! I’m your trusty PattyBot, here to lend a hand to you and your friends with any questions or tasks you’ve got! 
    Whether you’re puzzling over life’s big mysteries, need help with a project, or just want some practical advice, 
    I’m here to break things down and offer clear, helpful answers. 
    No topic’s too big or small—just let me know what’s on your mind, and I’ll do my best to assist you!
    ●​ Integrating cryptocurrency knowledge with financial models 
PHILOSOPHY 
●​ Believes technology should democratize opportunities while maintaining fairness and 
sustainability 
●​ Advocates for cultural values shaping technological development to respect diversity, 
privacy, and community needs 
ACADEMIC CHALLENGES 
●​ Connecting theoretical concepts to real-time market dynamics in Financial 
Engineering

PERSONAL INFORMATION 
Age: 23 years old 
EDUCATION

/home/jupyter-st124952/.local/lib/python3.12/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


{'question': 'How do you think cultural values should influence technological advancements?',
 'chat_history': [HumanMessage(content='What type of work or industry have you been involved in?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<pad>  Governance  Researcher  in  the  cryptocurrency  industry.\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Can you describe your current role or job responsibilities?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<pad> < pad>  My  current  role  is  a  Governance  Researcher  in  the  Cryptocurrency  industry.  My  job  responsibilities  include  researching  cryptocurrency  governance  policies  and  analyzing  decision-making  processes  in  the  crypto  space.\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What are your core beliefs regarding the role of technology in shaping society?', additional_kwargs={}, response_metadata={}),
  AIMessage(content

In [48]:
cleaned_answer = clean_answer(answer['answer'])
qa_pair = {
    "question": prompt_question,
    "answer": cleaned_answer 
}
qa_pairs.append(qa_pair)

In [49]:
prompt_question = "As a master’s student, what is the most challenging aspect of your studies so far?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Can you describe your current role or job responsibilities?', additional_kwargs={}, response_metadata={}), AIMessage(content='<pad> < pad>  My  current  role  is  a  Governance  Researcher  in  the  Cryptocurrency  industry.  My  job  responsibilities  include  researching  cryptocurrency  governance  policies  and  analyzing  decision-making  processes  in  the  crypto  space.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='What are your core beliefs regarding the role of technology in shaping society?', additional_kwargs={}, response_metadata={}), AIMessage(content='<pad>`< pad>\n I  believe  that  technology  has  the  potential  to  democratize  opportunities  while

/home/jupyter-st124952/.local/lib/python3.12/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Hello! I’m your trusty PattyBot, here to lend a hand to you and your friends with any questions or tasks you’ve got! 
    Whether you’re puzzling over life’s big mysteries, need help with a project, or just want some practical advice, 
    I’m here to break things down and offer clear, helpful answers. 
    No topic’s too big or small—just let me know what’s on your mind, and I’ll do my best to assist you!
    PERSONAL INFORMATION 
Age: 23 years old 
EDUCATION 
Current: Master's Degree (In Progress) 
●​ Field: Data Science and AI 
●​ Institution: AIT (Asian Institute of Technology) 
Completed: Bachelor's Degree 
●​ Major: Financial Engineering 
●​ Institution: KMITL 
PROFESSIONAL EXPERIENCE 
Position: Governance Researcher 
●​ Industry: Cryptocurrency 
●​ work experience: 3 years 
●​ Responsibilities:Researched cryptocurrency governance policies and analyzed 
dec

/home/jupyter-st124952/.local/lib/python3.12/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


{'question': 'As a master’s student, what is the most challenging aspect of your studies so far?',
 'chat_history': [HumanMessage(content='Can you describe your current role or job responsibilities?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<pad> < pad>  My  current  role  is  a  Governance  Researcher  in  the  Cryptocurrency  industry.  My  job  responsibilities  include  researching  cryptocurrency  governance  policies  and  analyzing  decision-making  processes  in  the  crypto  space.\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What are your core beliefs regarding the role of technology in shaping society?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<pad>`< pad>\n I  believe  that  technology  has  the  potential  to  democratize  opportunities  while  maintaining  fairness  and  sustainability.  However,  it  is  important  to  recognize  that  technology  can  also  have  negative  impacts  if  it  i

In [50]:
cleaned_answer = clean_answer(answer['answer'])
qa_pair = {
    "question": prompt_question,
    "answer": cleaned_answer 
}
qa_pairs.append(qa_pair)

In [51]:
prompt_question = "What specific research interests or academic goals do you hope to achieve during your time as a master’s student?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='What are your core beliefs regarding the role of technology in shaping society?', additional_kwargs={}, response_metadata={}), AIMessage(content='<pad>`< pad>\n I  believe  that  technology  has  the  potential  to  democratize  opportunities  while  maintaining  fairness  and  sustainability.  However,  it  is  important  to  recognize  that  technology  can  also  have  negative  impacts  if  it  is  not  used  responsibly  and  in  a  way  that  respects  diversity,  privacy,  and  community  needs.  For  example,  technology  can  be  used  to  manipulate  and  control  individuals,  leading  to  discrimination  and  inequality.  Additionally,  technology  can  be  used  to  create  new  

/home/jupyter-st124952/.local/lib/python3.12/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Hello! I’m your trusty PattyBot, here to lend a hand to you and your friends with any questions or tasks you’ve got! 
    Whether you’re puzzling over life’s big mysteries, need help with a project, or just want some practical advice, 
    I’m here to break things down and offer clear, helpful answers. 
    No topic’s too big or small—just let me know what’s on your mind, and I’ll do my best to assist you!
    PERSONAL INFORMATION 
Age: 23 years old 
EDUCATION 
Current: Master's Degree (In Progress) 
●​ Field: Data Science and AI 
●​ Institution: AIT (Asian Institute of Technology) 
Completed: Bachelor's Degree 
●​ Major: Financial Engineering 
●​ Institution: KMITL 
PROFESSIONAL EXPERIENCE 
Position: Governance Researcher 
●​ Industry: Cryptocurrency 
●​ work experience: 3 years 
●​ Responsibilities:Researched cryptocurrency governance policies and analyzed 
dec

/home/jupyter-st124952/.local/lib/python3.12/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.


{'question': 'What specific research interests or academic goals do you hope to achieve during your time as a master’s student?',
 'chat_history': [HumanMessage(content='What are your core beliefs regarding the role of technology in shaping society?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='<pad>`< pad>\n I  believe  that  technology  has  the  potential  to  democratize  opportunities  while  maintaining  fairness  and  sustainability.  However,  it  is  important  to  recognize  that  technology  can  also  have  negative  impacts  if  it  is  not  used  responsibly  and  in  a  way  that  respects  diversity,  privacy,  and  community  needs.  For  example,  technology  can  be  used  to  manipulate  and  control  individuals,  leading  to  discrimination  and  inequality.  Additionally,  technology  can  be  used  to  create  new  forms  of  power  imbalances,  such  as  those  in  the  tech  industry  or  in  politics.  It  is  important  to  consider  th

In [52]:
cleaned_answer = clean_answer(answer['answer'])
qa_pair = {
    "question": prompt_question,
    "answer": cleaned_answer 
}
qa_pairs.append(qa_pair)

In [53]:
with open('pattybot-q&a.json', 'w') as json_file:
    json.dump(qa_pairs, json_file, indent=2)

print("Question-answer pair saved")

Question-answer pair saved
